In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import warnings
warnings.filterwarnings('ignore')

class AutomotiveDataAnalyzer:
    def __init__(self, df):
        """Initialize with a pandas DataFrame"""
        self.df = df
        self.cleaned_df = None
        self.analysis_results = {}

    def clean_data(self):
        """Comprehensive data cleaning"""
        df = self.df.copy()

        # Date conversion
        df['REPAIR_DATE'] = pd.to_datetime(df['REPAIR_DATE'], format='%d-%m-%Y', errors='coerce')

        # Numeric cleaning
        numeric_columns = ['REPAIR_AGE', 'KM', 'REPORTING_COST', 'TOTALCOST', 'LBRCOST']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Text cleaning
        text_columns = ['CAUSAL_PART_NM', 'GLOBAL_LABOR_CODE_DESCRIPTION', 'CORRECTION_VERBATIM',
                       'CUSTOMER_VERBATIM']
        for col in text_columns:
            df[col] = df[col].fillna('').str.strip()

        # Standardize categorical columns
        categorical_columns = ['PLATFORM', 'BODY_STYLE', 'PLANT', 'BUILD_COUNTRY', 'STATE']
        for col in categorical_columns:
            df[col] = df[col].fillna('Unknown').str.strip()

        self.cleaned_df = df
        return df
    def analyze_vehicle_data(self):
        """Analyze vehicle-specific information"""
        df = self.cleaned_df

        # Platform and body style analysis
        platform_analysis = {
            'platform_distribution': df['PLATFORM'].value_counts().to_dict(),
            'body_style_by_platform': pd.crosstab(df['PLATFORM'], df['BODY_STYLE']).to_dict(),
            'avg_repair_cost_by_platform': df.groupby('PLATFORM')['TOTALCOST'].mean().to_dict()
        }

        # Engine and transmission analysis
        powertrain_analysis = {
            'engine_types': df['ENGINE_DESC'].value_counts().to_dict(),
            'transmission_types': df['TRANSMISSION_DESC'].value_counts().to_dict(),
            'common_combinations': pd.crosstab(df['ENGINE_DESC'], df['TRANSMISSION_DESC']).to_dict()
        }

        return {'platform_analysis': platform_analysis, 'powertrain_analysis': powertrain_analysis}

    def analyze_repair_patterns(self):
        """Analyze repair patterns and costs"""
        df = self.cleaned_df

        repair_analysis = {
            'common_parts': df['CAUSAL_PART_NM'].value_counts().head(10).to_dict(),
            'labor_codes': df['GLOBAL_LABOR_CODE'].value_counts().head(10).to_dict(),
            'avg_repair_age': df['REPAIR_AGE'].mean(),
            'avg_vehicle_km': df['KM'].mean(),
            'cost_statistics': {
                'mean_total_cost': df['TOTALCOST'].mean(),
                'median_total_cost': df['TOTALCOST'].median(),
                'cost_by_platform': df.groupby('PLATFORM')['TOTALCOST'].mean().to_dict()
            }
        }

        return repair_analysis

    def analyze_customer_feedback(self):
        """Analyze customer complaints and feedback"""
        df = self.cleaned_df

        # Extract common themes from verbatim
        def extract_themes(text):
            themes = []
            keywords = {
                'quality': ['quality', 'poor', 'bad', 'issue'],
                'safety': ['safety', 'dangerous', 'risk'],
                'performance': ['performance', 'power', 'speed'],
                'comfort': ['comfort', 'uncomfortable', 'ergonomic']
            }

            text = str(text).lower()
            for theme, words in keywords.items():
                if any(word in text for word in words):
                    themes.append(theme)
            return themes

        df['feedback_themes'] = df['CUSTOMER_VERBATIM'].apply(extract_themes)

        feedback_analysis = {
            'complaint_distribution': df['COMPLAINT_CD'].value_counts().to_dict(),
            'common_themes': df['feedback_themes'].explode().value_counts().to_dict(),
            'feedback_by_platform': pd.crosstab(df['PLATFORM'],
                                              df['feedback_themes'].apply(lambda x: len(x) > 0)).to_dict()
        }

        return feedback_analysis

    def analyze_geographical_patterns(self):
        """Analyze geographical patterns in repairs"""
        df = self.cleaned_df

        geo_analysis = {
            'repairs_by_state': df['STATE'].value_counts().to_dict(),
            'repairs_by_region': df['DEALER_REGION'].value_counts().to_dict(),
            'avg_cost_by_region': df.groupby('DEALER_REGION')['TOTALCOST'].mean().to_dict(),
            'dealer_distribution': {
                'dealers_per_state': df.groupby('STATE')['REPAIRING_DEALER_CODE'].nunique().to_dict(),
                'repairs_per_dealer': df['REPAIRING_DEALER_CODE'].value_counts().describe().to_dict()
            }
        }

        return geo_analysis

    def create_visualizations(self):
        """Generate comprehensive visualizations"""
        df = self.cleaned_df

        # 1. Repair Costs Distribution
        plt.figure(figsize=(10, 6))
        sns.histplot(data=df, x='TOTALCOST', bins=50)
        plt.title('Distribution of Total Repair Costs')
        plt.xlabel('Total Cost')
        plt.ylabel('Count')
        plt.savefig('repair_costs_distribution.png')
        plt.close()

        # 2. Repairs by Platform and Body Style
        plt.figure(figsize=(12, 8))
        platform_body = pd.crosstab(df['PLATFORM'], df['BODY_STYLE'])
        sns.heatmap(platform_body, annot=True, fmt='d', cmap='YlGnBu')
        plt.title('Repairs by Platform and Body Style')
        plt.tight_layout()
        plt.savefig('platform_body_repairs.png')
        plt.close()

        # 3. Geographical Distribution
        plt.figure(figsize=(15, 7))
        df['STATE'].value_counts().plot(kind='bar')
        plt.title('Repairs by State')
        plt.xlabel('State')
        plt.ylabel('Number of Repairs')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('repairs_by_state.png')
        plt.close()

        # 4. Repair Age vs Cost
        plt.figure(figsize=(10, 6))
        plt.scatter(df['REPAIR_AGE'], df['TOTALCOST'], alpha=0.5)
        plt.title('Repair Age vs Total Cost')
        plt.xlabel('Repair Age')
        plt.ylabel('Total Cost')
        plt.savefig('repair_age_vs_cost.png')
        plt.close()

    def generate_report(self):
        """Generate comprehensive analysis report"""
        report = {
            'summary_statistics': {
                'total_records': len(self.cleaned_df),
                'date_range': f"{self.cleaned_df['REPAIR_DATE'].min()} to {self.cleaned_df['REPAIR_DATE'].max()}",
                'total_cost': self.cleaned_df['TOTALCOST'].sum(),
                'average_cost': self.cleaned_df['TOTALCOST'].mean(),
                'unique_vehicles': self.cleaned_df['VIN'].nunique(),
                'unique_dealers': self.cleaned_df['REPAIRING_DEALER_CODE'].nunique()
            },
            'vehicle_analysis': self.analyze_vehicle_data(),
            'repair_analysis': self.analyze_repair_patterns(),
            'customer_feedback': self.analyze_customer_feedback(),
            'geographical_analysis': self.analyze_geographical_patterns()
        }

        return report

    def export_results(self, filename_prefix):
        """Export analysis results"""
        # Export cleaned data
        self.cleaned_df.to_csv(f'{filename_prefix}_cleaned_data.csv', index=False)

        # Export analysis report
        report = self.generate_report()
        with open(f'{filename_prefix}_analysis_report.json', 'w') as f:
            import json
            json.dump(report, f, indent=4)

        # Generate visualizations
        self.create_visualizations()

def main():
    # Read the data

    df = pd.read_excel('Data_for_Task_1.xlsx')

    # Initialize analyzer
    analyzer = AutomotiveDataAnalyzer(df)

    # Clean data
    analyzer.clean_data()

    # Run analysis and export results
    analyzer.export_results('automotive_repair')

if __name__ == "__main__":
    main()
